In [1]:
import pandas as pd
import numpy as np
import datetime
import ast

from sqlalchemy import create_engine

In [11]:
# Indices details as ticker, market...
credentials = ast.literal_eval(open(r"C:\Users\pvilas\OneDrive - unizar.es\Python\credentials.txt", "r").read())
sql_engine = create_engine(credentials['sqlalchemy'][0]+ 'market_index')

equity_indices_detail=pd.read_excel(r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\Artículo 3 Sustainability Journal\Factsheet\index_details.xlsx')
equity_indices_detail.to_sql('index_details', sql_engine, if_exists="replace", index=False)

In [9]:
# indices constituents to database
# change numeric date to datetime
def obtain_date(numeric_date):    
    numeric_date = str(numeric_date)
    numeric_date = datetime.datetime(int(numeric_date[0:4]),int(numeric_date[4:6]),1)
    return numeric_date

equity_indices_details = pd.read_sql('index_details', sql_engine)
for row in range(0, len(equity_indices_details)):
    index_ticker = equity_indices_details.at[row,'index_ticker']
    # we read each listado with the information of the index
    df_index = pd.read_excel(r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\Artículo 3 Sustainability Journal\1. Data\Indices Composition\%s.xlsx' % index_ticker)
    df_index['index_ticker'] = index_ticker # We add a column with the ticker of the index
    df_index = df_index.loc[:, ['index_ticker', 'Fecha Numerica', 'Type', 'DSCD', 'ISIN', 'RIC', 'WC06035', 'WC05601', 'NAME',
                                'WTIDX', 'NOSHFF']] # change order of columns
    df_index.columns = ['index_ticker', 'monthly_date', 'refinitiv_type', 'datastream_id', 'isin_code', 'ric_code', 'worldscope_code',
                       'ticker', 'ds_name', 'company_weight','free_float' ] # change the name
    df_index['monthly_date'] = df_index['monthly_date'].apply(obtain_date) # numeric date to datetime
    # we append the columns to the databse
    df_index.to_sql('index_constituents', sql_engine, index=False, 
                    if_exists="append", chunksize=100000, method='multi' )
    print(index_ticker)
    

LFT4GBGL
LFT4G100
LAWDVLP
LFT4GBEU
LAWDVER
LFT4GBUS
LFT4U100
LWIUSAM


In [ ]:
monthly_scores = pd.read_excel(r"C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\Excel central con  tablas de trabajo.xlsm",
                          usecols ="B:D, R:U, Y" )

all_scores.columns = ['MonthlyDate', 'DsName', 'IsinCode', 'MarketValueUsd', 'EsgScore', 'CombinedScore',
       'ControversiesScore', 'WcNation']

all_scores.to_sql('CompaniesMonthlyData', sql_engine, chunksize=100000, method='multi', index=False)

In [13]:
geo_classification = pd.read_excel(r"C:\Users\pablo\OneDrive - unizar.es\Artículo 1\4.1. Descarga capitalización y sectores de las empresas ESG Reuters (7.469)\Excel central con  tablas de trabajo.xlsm",
                          sheet_name ="Info Títulos" )